In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.under_sampling import RandomUnderSampler, TomekLinks, NearMiss
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier


url = 'https://raw.githubusercontent.com/AnjulaMehto/Sampling_Assignment/main/Creditcard_data.csv'
data = pd.read_csv(url)


X = data.drop('Class', axis=1)
y = data['Class']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


conf_interval = 1.96  
error_margin = 0.05  


n1 = int(np.ceil((conf_interval**2 * 0.5 * 0.5) / (error_margin**2)))
n2 = int(np.ceil((conf_interval**2 * 0.05 * (1-0.05)) / (error_margin**2)))
n3 = int(np.ceil((conf_interval**2 * 0.05 * (1-0.05)) / (error_margin**2)))
n4 = int(np.ceil((conf_interval**2 * 0.05 * (1-0.05)) / (error_margin**2)))
n5 = int(np.ceil((conf_interval**2 * 0.05 * (1-0.05)) / (error_margin**2)))


samp1 = RandomUnderSampler(sampling_strategy='majority', random_state=42)
samp2 = RandomOverSampler(sampling_strategy='minority', random_state=42)
samp3 = SMOTE(sampling_strategy='minority', random_state=42)
samp4 = TomekLinks(sampling_strategy='majority')
samp5 = NearMiss(version=3, n_neighbors=3)

mod1 = GradientBoostingClassifier()
mod2 = DecisionTreeClassifier(random_state=42)
mod3 = AdaBoostClassifier(n_estimators=100, random_state=42)
mod4 = SVC(random_state=42)
mod5 = GaussianNB()


samplers = {
    'Sampling1': samp1,
    'Sampling2': samp2,
    'Sampling3': samp3,
    'Sampling4': samp4,
    'Sampling5': samp5,
}
models = {
    'M1': mod1,
    'M2': mod2,
    'M3': mod3,
    'M4': mod4,
    'M5': mod5,
}


results = {}
for sampler_name, sampler in samplers.items():
    if sampler_name == 'Sampling1':
        n = n1
    elif sampler_name == 'Sampling2':
        n = n2
    elif sampler_name == 'Sampling3':
        n = n3
    elif sampler_name == 'Sampling4':
        n = n4
    else:
        n = n5

    
    X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
    
    
    if len(X_resampled) > n:
        X_resampled = X_resampled[:n]
        y_resampled = y_resampled[:n]
    
    for model_name, model in models.items():
        
        model.fit(X_resampled, y_resampled)
        
        
        y_pred = model.predict(X_test)
        
        
        accuracy = accuracy_score(y_test, y_pred)
        
        
        if model_name in results:
            results[model_name][sampler_name] = accuracy
        else:
            results[model_name] = {sampler_name: accuracy}
            

print('Results:')
print('        Sampling1   Sampling2   Sampling3   Sampling4   Sampling5')
for model_name, model_results in results.items():
    print(model_name, end='')
    for sampler_name in samplers.keys():
        if sampler_name in model_results:
            print(f'    {model_results[sampler_name]:.4f}   ', end='')
        else:
            print('              ', end='')
    print() 




Results:
        Sampling1   Sampling2   Sampling3   Sampling4   Sampling5
M1    0.5742       0.9806       0.9742       0.9806       0.8387   
M2    0.5742       0.9806       0.9806       0.9806       0.6968   
M3    0.5742       0.9935       0.9935       0.9935       0.6194   
M4    0.6258       0.9935       0.9935       0.9935       0.5161   
M5    0.6968       0.9935       0.9935       0.9935       0.4839   
